In [10]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import cross_validate
from collections import defaultdict

In [8]:
# path to dataset file
file_path = "../ALS/data/ratings.csv"

reader = Reader(line_format= "user item rating", sep=",", skip_lines=1)

data = Dataset.load_from_file(file_path=file_path, reader=reader)

In [9]:
algo = SVD()

# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8305  0.8309  0.8298  0.8296  0.8313  0.8304  0.0006  
MAE (testset)     0.6411  0.6415  0.6405  0.6406  0.6415  0.6410  0.0004  
Fit time          23.82   23.01   23.73   22.96   23.78   23.46   0.39    
Test time         6.85    5.04    6.72    5.97    5.51    6.02    0.69    


{'test_rmse': array([0.83045064, 0.83087787, 0.82979055, 0.82958435, 0.83125526]),
 'test_mae': array([0.64106566, 0.64145378, 0.64050602, 0.64063575, 0.64150628]),
 'fit_time': (23.821056127548218,
  23.014048099517822,
  23.731786966323853,
  22.955167055130005,
  23.782928943634033),
 'test_time': (6.845360040664673,
  5.040719032287598,
  6.719989061355591,
  5.972475290298462,
  5.506942987442017)}

In [11]:

def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [ ]:
# Train an SVD using test and split dataset
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])